In [11]:
import numpy as np
import time, os, sys
import mxnet as mx
import matplotlib.pyplot as plt
import skimage
import glob
import sys
import pandas as pd
#sys.path.insert(0,'/github/cellpose/')
from cellpose import models, utils
import cv2

In [12]:
CSV_FOLDER = 'data/raw/CSV_patryk/'
TIFF_FOLDER = 'data/raw/RAW_TIFF/'
TEST_IMAGE = 'data/raw/RAW_TIFF/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_10_ch_4.tif'
TEST_CSV = 'data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_10_ch_4.csv'
BLOCK_ANNOTATION = 'data/raw/block_annotation2.csv'
BLOCK_SIZE = 50
TO_IGNORE = 'ND8_DIV0+2h_'

In [13]:
use_gpu = utils.use_gpu()
if use_gpu:
    device = mx.gpu()
    print("use gpu")
else:
    device = mx.cpu()

use gpu


In [25]:
def processAll(folder_csv,folder_tiff):
    X = []
    y = []
    all_csv,all_tiff = prepareList(folder_csv,folder_tiff,TO_IGNORE)
#     all_csv= all_csv[:2]
#     all_tiff= all_tiff[:2]
    annotation = pd.read_csv(BLOCK_ANNOTATION, header=None)
    all_mask = getMasks(all_tiff)
    save(all_mask,all_tiff)
    for csv, tiff, mask in zip(all_csv, all_tiff,all_mask):
        
        Xt, yt = processImage(csv,tiff,mask,annotation)
        X = X + Xt
        y = y + yt
    return X,y
 
def save(all_mask,all_tiff):
    for i in range(len(all_tiff)):
        name = os.path.basename(all_tiff[0]).replace('.tif','.jpg')
        cv2.imwrite('data/masks/%s'%name, all_mask[i])

def processImage(csv,tiff,mask,annotations):
    print('Processing %s'%os.path.basename(tiff))
    Xt = []
    yt = []
    data = pd.read_csv(csv)
    size = data.shape[0]
    vals = getVals(data,mask)
    image = plt.imread(tiff)
    base_name = os.path.basename(tiff).split('.')[0].replace("_ch_4", "")
    for i in range(size):
        x,y = getPoint(data,i)
        val = vals[i]
        block_name = '%s_%d'%(base_name,i)
        annotation = getAnnotation(annotations,block_name)
        if annotation == -1 :
            continue
        if val > 0 :
            activated_mask = activate(mask,val)
            activated_mask = np.stack((activated_mask,)*3, axis=-1)
            masked_image = activateImage(image, activated_mask)
            height, width = image.shape
            if(checkPosition(height, width,y,x,BLOCK_SIZE)==False):
                continue
            cropped_image = cropImage(masked_image,x,y,BLOCK_SIZE)
            Xt.append(cropped_image)
            yt.append(annotation)
    print('%s created %d images'%(os.path.basename(tiff),len(Xt)))
    return Xt, yt

def getAnnotation(annotations, name):
    for file,val in zip(annotations[0],annotations[1]):
        if name in file:
            return val
    return -1
        
def checkPosition(height,width,y,x,size):
    if x<size :
        return False
    if y<size :
        return False
    if x+size>width :
        return False
    if y+size>height:
        return False
    return True

def cropImage(image,x,y,size):
    height, width = image.shape
    x1 = max([0,x-size+1])
    y1 = max([0,y-size+1])
    x2 = min([width,x+size])
    y2 = min([height,y+size])
#     base=np.zeros((size*2,size*2),dtype=np.uint8)
    return image[y1:y2, x1:x2]
    
def show(img,title):
    fig2 = plt.figure(figsize = (15,15))
    ax3 = fig2.add_subplot(111)
    ax3.imshow(img, interpolation='none')
    ax3.set_title(title)

def getPoint(data,i):
    return int(data['x'][i]),int(data['y'][i])

def activate(array,value):
    return (array == value) * 1

def activateImage(image,mask):
    return (mask == 1)*image

def getVals(data,mask):
    vals = []
    not_found = 0
    duplicate = 0
    for i in range(data.shape[0]):
        x,y = getPoint(data,i)
        val = mask[y][x]
        if val == 0 :
            not_found = not_found +1
            print("%d- %d Not found!"%(i,val))
        else:
            if val in vals:
                duplicate = duplicate +1
                print("%d- %d already exists!"%(i,val))
        vals.append(val)
    print("not found: %d"%not_found)
    print("duplicate: %d"%duplicate)
    return vals,not_found,duplicate

    
def prepareList(folder_csv,folder_tiff,to_ignore):
    all_csv = []
    all_tiff = []
    list_tiff = os.listdir(folder_tiff)
    
    for tiff in list_tiff:
        tiff_path = os.path.join(folder_tiff,tiff)
        csv = "%s.csv"%tiff.split('.')[0]
        csv_path = os.path.join(folder_csv,csv)
        if not os.path.exists(tiff_path):
            print('%s not exist'%tiff)
            continue
        if not os.path.exists(csv_path):
            print('%s not exist'%csv)
            continue
        if to_ignore in tiff :
            print('%s to ignore'%csv)
            continue
        all_csv.append(csv_path)
        all_tiff.append(tiff_path)
    return all_csv,all_tiff

def getMasks(list_tiff):
    print('Detecting masks ..')
    model = models.Cellpose(device, model_type='cyto')
    imgs = [plt.imread(f) for f in list_tiff]
    nimg = len(imgs)
    channel =[2,1]
    channels = [channel for i in range(len(list_tiff))]
    masks, _, _, _ = model.eval(imgs, rescale=None, channels=channels)
    return masks



In [26]:
all_csv,all_tiff = prepareList(CSV_FOLDER,TIFF_FOLDER,TO_IGNORE)
print("CSV")
print(all_csv)
print("TIFF")
print(all_tiff)

ND8_DIV0+2h_3_ch_4.csv to ignore
ND8_DIV0+2h_9_ch_4.csv to ignore
ND8_DIV0+2h_7_ch_4.csv to ignore
ND8_DIV0+2h_2_ch_4.csv to ignore
ND8_DIV0+2h_6_ch_4.csv to ignore
ND8_DIV0+2h_5_ch_4.csv to ignore
ND8_DIV0+2h_1_ch_4.csv to ignore
ND8_DIV0+2h_10_ch_4.csv to ignore
ND8_DIV0+4h_20x_noConfinment_2_ch_4.csv not exist
ND8_DIV0+2h_4_ch_4.csv to ignore
ND8_DIV0+2h_11_ch_4.csv to ignore
ND8_DIV0+2h_8_ch_4.csv to ignore
CSV
['data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_26_ch_4.csv', 'data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_6_ch_4.csv', 'data/raw/CSV_patryk/ND8_DIV0+4h_20x_noConfinment_5_ch_4.csv', 'data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_9_ch_4.csv', 'data/raw/CSV_patryk/ND8_DIV0+4h_20x_noConfinment_7_ch_4.csv', 'data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_1_ch_4.csv', 'data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_19_ch_4.csv', 'data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_31_

In [28]:
test = 'data/raw/CSV_patryk/20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_26_ch_4.csv'
test = pd.read_csv(test, header=None)
test

,0,1
0,x,y
1,369.7574449719465,17.949287872248597
2,954.5366182014001,21.486268174474958
3,392.15694940756066,27.102031220613128
4,475.90909090909093,27.801929913661755
...,...,...
220,202.9950542134297,981.4324709910595
221,46.11341686243801,993.3086661446098
222,28.549728117874057,1001.8709436940887
223,644.3860810810811,1000.2891891891892


In [27]:
annotation = pd.read_csv(BLOCK_ANNOTATION, header=None)
annotation

,0,1
0,ND8_DIV0+4h_20x_noConfinment_6_ch_4_30.tif,2
1,20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_32_...,4
2,20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_31_...,4
3,20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_28_...,4
4,20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_10_...,0
...,...,...
2277,20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_23_...,3
2278,ND8_DIV0+4h_20x_noConfinment_7_ch_4_97.tif,2
2279,ND8_DIV0+4h_20x_noConfinment_4_ch_4_186.tif,2
2280,20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_31_...,4


In [21]:
X,y = processAll(CSV_FOLDER,TIFF_FOLDER)
print(X.shape)
print(y.shape)

ND8_DIV0+2h_3_ch_4.csv to ignore
Detecting masks ..
Processing 20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo_26_ch_4.tif
0- 0 Not found!
5- 0 Not found!
6- 0 Not found!
7- 7 already exists!
8- 0 Not found!
9- 0 Not found!
10- 0 Not found!
11- 0 Not found!
14- 0 Not found!
16- 0 Not found!
19- 0 Not found!
21- 0 Not found!
22- 0 Not found!
28- 21 already exists!
31- 0 Not found!
35- 24 already exists!
36- 0 Not found!
37- 0 Not found!
44- 0 Not found!
48- 0 Not found!
49- 0 Not found!
52- 0 Not found!
53- 0 Not found!
56- 0 Not found!
57- 0 Not found!
58- 0 Not found!
60- 0 Not found!
61- 0 Not found!
62- 0 Not found!
64- 0 Not found!
73- 0 Not found!
75- 0 Not found!
77- 0 Not found!
80- 0 Not found!
82- 0 Not found!
83- 0 Not found!
86- 0 Not found!
89- 0 Not found!
99- 0 Not found!
103- 0 Not found!
110- 0 Not found!
111- 0 Not found!
123- 0 Not found!
124- 0 Not found!
137- 0 Not found!
152- 0 Not found!
153- 0 Not found!
159- 0 Not found!
161- 0 Not found!
162- 0 Not found!
163- 0 Not

IndexError: tuple index out of range

In [ ]:
X = np.array(X).reshape(-1,100,100,3)
y = np.array(y).reshape(-1,1)



pickle_out = open("data/X2_masked.pickle","wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("data/y2_masked.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

print(X.shape) 
print(y.shape)

In [ ]:
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:


# for idx in range(nimg):
#     img = transforms.reshape(imgs[idx], channels[idx])
#     img = plot.rgb_image(img)
#     maski = masks[idx]
#     flowi = flows[idx][0]

#     fig = plt.figure(figsize=(12,3))
#     # can save images (set save_dir=None if not)
#     plot.show_segmentation(fig, img, maski, flowi)
#     plt.tight_layout()
#     plt.show()

In [ ]:
# show(imgs[0],'Original')
# show(masks[0],'Mask')
# show(flows[0][0],'Flow')

In [ ]:
# masks[0].max()

In [ ]:
# activated_mask = activate(masks[0],110)
# print(activated_mask.max())
# activated_mask = np.stack((activated_mask,)*3, axis=-1)
# print(activated_mask.shape)
# show(activated_mask,'Mask')

In [ ]:
# import cv2
# image = imgs[0]
# dst = activateImage(image, stacked_img)
# show(dst,'Mask')

In [ ]:
# number_segments = masks[0].max()
# # number_segments
# for i in range(1,20):
#     x = activate(masks[0],i)
#     stacked_img = np.stack((x,)*3, axis=-1)
#     dst = activateImage(image, stacked_img)
#     show(dst,str(i))

In [9]:
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
Channels_map = {
        '20072019_ND9_ND11_DIV0-1_Daam1_aTub_Phallo': [3,2],
        'ND8_DIV0+2h': [2,1],
        'ND8_DIV0+4h_20x_noConfinment': [3,1]
    }

def getChannels(name):
    for key, channels in Channels_map.items():
        if name in key:
            return channels
    print('Name not found, return default [2,3]')
    return [2,3]
        
getChannels('20072019_ND9_ND11_DIV0-1_Daam1')

[3, 2]